In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!pip install transformers==2.5.1
!pip install simpletransformers==0.20.3
!pip install seqeval==0.0.12
!pip install torch==1.4+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html --no-cache-dir
!rm -rf .git
!rm -rf apex
!git clone https://github.com/NVIDIA/apex
!git checkout 494f8ab3fc1b0b26949a3bcbb2bcac78008d48c1
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
cd /content/drive/My\ Drive/Colab\ Notebooks

In [ ]:
import numpy as np
import pandas as pd
import torch
import gc
import csv

from CsvReader import ReadCsv
from HelperFunctions import ShuffleData, replace_all, ReplaceSentimentsWithIndexes, SplitArticlesIntoSentiments, getMetrics, TransformDataIntoDataframe, CalculateWeights
from TrainEvalModel import TrainModelForMultiClass, EvalFromModel

path = '/content/drive/My Drive/Colab Notebooks Pascal/bert-etiki/etiki-data'
data, categories, tendencies = ReadCsv(path, 'test.csv','companies.csv', 'categories.csv','references.csv','tendencies.csv', 'topics.csv')
sentimentData = ReplaceSentimentsWithIndexes(data[:,[0,7,13]])
labels = np.unique(sentimentData[:,1])
weights = CalculateWeights(labels,sentimentData)

for j in range(2,6):
  positiveArticles, negativeArticles, controversialArticles = SplitArticlesIntoSentiments(sentimentData)
  positiveArticles = ShuffleData(positiveArticles)
  negativeArticles = ShuffleData(negativeArticles)
  controversialArticles = ShuffleData(controversialArticles)
  trainData = np.vstack((positiveArticles[:int(len(positiveArticles)*0.7)],controversialArticles[:int(len(controversialArticles)*0.7)],negativeArticles[:int(len(negativeArticles)*0.7)]))
  testData = np.vstack((positiveArticles[int(len(positiveArticles)*0.7):],controversialArticles[int(len(controversialArticles)*0.7):],negativeArticles[int(len(negativeArticles)*0.7):]))
  train_df = TransformDataIntoDataframe(trainData)
  eval_df = TransformDataIntoDataframe(testData)
#-----------------------------128------------------------------
  algo = 'xlnet'        
  args = {'reprocess_input_data': True,
            'overwrite_output_dir': True,
            'num_train_epochs':j ,
            'silent':True,
            'use_cached_eval_features':True, 
            'max_seq_length':128
            }
  model = TrainModelForMultiClass(algo, 'xlnet-base-cased',train_df,3,args, weights)
  result, model_outputs, wrong_predictions = EvalFromModel(model,eval_df)
  with open('results/'+str(j)+'epoch/'+algo+'-mcc.csv', 'a', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([result['mcc'],result["acc"],result["eval_loss"]])
  getMetrics(result["matr"],3,str(j)+'epoch/'+algo) 
#-----------------------------256------------------------------
  algo = 'xlnet'        
  args = {'reprocess_input_data': True,
            'overwrite_output_dir': True,
            'num_train_epochs':j ,
            'silent':True,
            'use_cached_eval_features':True, 
            'max_seq_length':256
            }
  model = TrainModelForMultiClass(algo, 'xlnet-base-cased',train_df,3,args, weights)
  result, model_outputs, wrong_predictions = EvalFromModel(model,eval_df)
  with open('results/'+str(j)+'epoch/'+algo+'-mcc.csv', 'a', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([result['mcc'],result["acc"],result["eval_loss"]])
  getMetrics(result["matr"],3,str(j)+'epoch/'+algo)
  #-----------------------------512------------------------------
  algo = 'xlnet'        
  args = {'reprocess_input_data': True,
            'overwrite_output_dir': True,
            'num_train_epochs':j ,
            'silent':True,
            'use_cached_eval_features':True, 
            'max_seq_length':512
            }
  model = TrainModelForMultiClass(algo, 'xlnet-base-cased',train_df,3,args, weights)
  result, model_outputs, wrong_predictions = EvalFromModel(model,eval_df)
  with open('results/'+str(j)+'epoch/'+algo+'-mcc.csv', 'a', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([result['mcc'],result["acc"],result["eval_loss"]])
  getMetrics(result["matr"],3,str(j)+'epoch/'+algo)